### Long memory

Generate datasets

In [51]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_json("msc_self_instruct.jsonl", lines=True)

answer_df = pd.DataFrame()
answer_df['dialog'] = None
answer_df['question'] = None
answer_df['gold_answer'] = None
answer_df['long_mem_result'] = None
answer_df['long_mem_answer'] = None
answer_df['long_mem_f1'] = None
answer_df['long_mem_rc'] = None
answer_df['long_mem_pre'] = None
answer_df['long_mem_recall_result'] = None
answer_df['long_mem_recall_answer'] = None
answer_df['long_mem_recall_f1'] = None
answer_df['long_mem_recall_rc'] = None
answer_df['long_mem_recall_pre'] = None

for i in range(500):
    # Add dialog
    current_time = datetime.strptime("2024/11/1 12:00", "%Y/%m/%d %H:%M")
    dialog_data = []
    for dialog_session in df['previous_dialogs'][i]:
        session = []
        for count in range(int(len(dialog_session['dialog'])/2)):
            chat_log = {"text":f"user:{dialog_session['dialog'][2*count]['text']}, assistant:{dialog_session['dialog'][2*count+1]['text']}", "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")}
            session.append(chat_log)
            current_time = current_time + timedelta(minutes=1)
        dialog_data.append(session)
    answer_df.loc[i, 'dialog'] = str(dialog_data)
    
    # Add question & answer
    answer_df.loc[i, 'question'] = f"{df['self_instruct'][i]['B']}"
    answer_df.loc[i, 'gold_answer'] = f"{df['self_instruct'][i]['A']}"
answer_df.to_json("MSC_datasets.json", orient="records", lines=True)

In [1]:
import pandas as pd

answer_df = pd.read_json("MSC_datasets.json", lines=True)
# answer_df = pd.read_json("MSC_eval.json", lines=True)

In [2]:
from long_memory.component import WeaviateLongMemory
long_mem = WeaviateLongMemory()

Detect existed deafult user group memory space, loading...
Detect existed deafult user child memory space, loading...


In [3]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )
        return completion.choices[0].message.content

classify_instruct="""The summary should focus on keywords such as personal preferences, experiences, activities, and locations."""

recall_instruct="""You will get a question about user. find relative memory as more as possible, don't give up easily"""

answer_prompt = """Base on the following document and answer the question.
If you are User, what will you say, give a brief answer

Question:{question}

Document:{docs}
"""

In [18]:
error_list = []
error_log = []
for i in range(1):
    print(f'==={i}===')
    try:
        # generate result
        long_mem.del_memory()
        dialogs = []
        for dialog_session in eval(answer_df.loc[i, 'dialog']):
            dialogs.extend(dialog_session)
        long_mem.add_chat_logs(dialogs, other_instruct=classify_instruct)
        question = f"{answer_df.loc[i, 'question']}"
        answer_df.loc[i, 'long_mem_result'] = str(long_mem.get_memory(question, recall=False))
        answer_df.loc[i, 'long_mem_recall_result'] = str(long_mem.get_memory(question, recall=True, other_instruct=recall_instruct))
        
        # response
        p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_result'])
        long_mem_answer = llm_create(p)
        answer_df.loc[i, 'long_mem_answer'] = long_mem_answer
        
        p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_recall_result'])
        long_mem_recall_answer = llm_create(p)
        answer_df.loc[i, 'long_mem_recall_answer'] = long_mem_recall_answer
    except Exception as e:
        error_list.append(i)
        error_log.append(e)
        print(f'----error:{e}----')
    if (i+1)%20==0:
        answer_df.to_json("MSC_eval.json", orient="records", lines=True)
answer_df.to_json("MSC_eval.json", orient="records", lines=True)

===1===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===4===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===15===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Chat logs not correct, missing id:{3}, unknown id:set(), retry..
Save chat logs to long memory done.
===19===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===23===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===33===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===34===
Detect empty group memory, create memory space...
Detect e

In [ ]:
# prompt 新增兩者可以互相紀錄, 可以不只一組

In [ ]:
long_mem.get_memory('Hey, remember that time we talked about your concerts? What was that specific color of the hat you mentioned you later on?', recall=True, other_instruct=recall_instruct)

In [8]:
error_list

[82]

In [9]:
long_mem.recall_search_record

{'Question': 'Hey, remember that time we talked about my grandma Frankie? What did I tell you we usually end up having for dinner on Thanksgiving at her place?',
 'records': [{'search_times': 1,
   'used_kewords': 'Hey, remember that time we talked about my grandma Frankie? What did I tell you we usually end up having for dinner on Thanksgiving at her place?',
   'searched memory': {'text': 'User shares travel experiences in Spain while the assistant expresses a lack of travel. They also discuss feelings regarding family Thanksgiving traditions.',
    'time': '2024/11/01 12:04'},
   'thought': "The information about Thanksgiving dinner at your grandma Frankie's house has been retrieved from the related discussions about family traditions.",
   'evdience': [{'text': "user:I've been to spain a few times, where do you go for thanksgiving?, assistant:My grandma frankie s house. My mom s mom who lives in ojai."},
    {'text': 'User shares travel experiences in Spain while the assistant expr

In [19]:
long_mem.show_group_and_children()

[{'group': 'User shares enjoyment of traveling, mentioning visits to Spain. Assistant reveals they do not travel much, usually staying home except for Thanksgiving.',
  'children': [{'text': "user:That's great, does your family like to travel? I do, assistant:No. Not too much. I take a school bus but we mostly stay at home. Except thanksgiving",
    'time': '2024/11/01 12:03'},
   {'text': "user:I've been to spain a few times, where do you go for thanksgiving?, assistant:My grandma frankie s house. My mom s mom who lives in ojai.",
    'time': '2024/11/01 12:04'},
   {'text': "user:What's it like in Spain?, assistant:I have never been to Spain, i though you ",
    'time': '2024/11/01 12:08'}]},
 {'group': "User inquires about Thanksgiving traditions. Assistant humorously notes their grandmother's cooking is bad, leading to meals from Pollo Loco.",
  'children': [{'text': 'user:That sounds like fun, with lots of good food, assistant:Not really. She s a bad cook and we end up getting pol

In [22]:
index=22
eval(answer_df['long_mem_recall_result'][index])

{'search_times': 2,
 'used_keywords': ['Hey, remember that time we talked about my job? Do you happen to recall how much I told you I make annually?'],
 'searched_memory': [{'text': "User asks about assistant's knowledge and relationship status. Assistant jokes about dislike for orange clothing and shares financial challenges.",
   'time': '2024/11/01 12:03'},
  {'text': "Discussion on student loans and job opportunities. User expresses support for assistant's job in research, while sharing pride in her husband's promotion to Chief of Surgery. Both emphasize the importance of taking one step at a time in their careers.",
   'time': '2024/11/01 12:15'}],
 'thought': "While there are discussions about jobs and financial challenges, the user's exact annual income is not disclosed in the retrieved information.",
 'evidence': [{'text': "Discussion on student loans and job opportunities. User expresses support for assistant's job in research, while sharing pride in her husband's promotion to

In [ ]:
eval(answer_df['long_mem_result'][index])

In [29]:
answer_df['gold_answer'][index]

"You mentioned you'd like to go to culinary school!"

### Short memory

In [ ]:
import pandas as pd

df = pd.read_json("msc_self_instruct.jsonl", lines=True)

s_answer_df = pd.DataFrame()
s_answer_df['dialog'] = None
s_answer_df['question'] = None
s_answer_df['gold_answer'] = None
s_answer_df['short_mem_result'] = None
s_answer_df['short_mem_answer'] = None
s_answer_df['short_mem_f1'] = None
s_answer_df['short_mem_rc'] = None
s_answer_df['short_mem_pre'] = None

for i in range(500):
    # Add dialog
    current_time = datetime.strptime("2024/11/1 12:00", "%Y/%m/%d %H:%M")
    dialog_data = []
    for dialog_session in df['previous_dialogs'][i]:
        session = []
        for count in range(int(len(dialog_session['dialog'])/2)):
            chat_log = {"assistant":f"{dialog_session['dialog'][2*count]['text']}", 
                        "user":f"{dialog_session['dialog'][2*count+1]['text']}", 
                        "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")}
            session.append(chat_log)
            current_time = current_time + timedelta(minutes=1)
        dialog_data.append(session)
    s_answer_df.loc[i, 'dialog'] = str(dialog_data)
    
    # Add question & answer
    s_answer_df.loc[i, 'question'] = f"user:{df['self_instruct'][i]['B']}"
    s_answer_df.loc[i, 'gold_answer'] = f"{df['self_instruct'][i]['A']}"
s_answer_df.to_json("s_MSC_datasets.json", orient="records", lines=True)

In [ ]:
import pandas as pd

s_answer_df = pd.read_json("s_MSC_datasets.json", lines=True)
# s_answer_df = pd.read_json("s_MSC_eval.json", lines=True)

In [ ]:
from short_memory.component import WeaviateShortMemory
short_mem = WeaviateShortMemory()

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content

answer_prompt = """Base on the following document and answer the question.
Try use origin text in the evidence field to repeat a brief answer.

Question:{question}

Document:{docs}
"""

In [ ]:
error_list = []
for i in range(100):
    print(f'==={i}===')
    try:
        # generate result
        short_mem.del_memory()
        dialogs = []
        for dialog_session in eval(s_answer_df.loc[i, 'dialog']):
            dialogs.extend(dialog_session)
        short_mem.add_chatlogs(dialogs)
        question = f"{answer_df.loc[i, 'question']}"
        s_answer_df.loc[i, 'short_mem_result'] = str(short_mem.get_memory(question))
        
        # response
        p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'short_mem_result'])
        short_mem_answer = llm_create(p)
        s_answer_df.loc[i, 'short_mem_answer'] = short_mem_answer
    except Exception as e:
        error_list.append(i)
        print(f'----error:{e}----')
    if (i+1)%20==0:
        s_answer_df.to_json("s_MSC_eval.json", orient="records", lines=True)
s_answer_df.to_json("s_MSC_eval.json", orient="records", lines=True)